In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import datasets
from scipy import stats

# Functions

In [ ]:
def Scale(mat, scaleMin, scaleMax):
    """
    Function expects a matrix along with minimum and maximum values 
    between which all the values of matrix are scaled.
    It returns a scaled matrix.
    """
    oldMin   = mat.min()
    oldRange = mat.max() - mat.min()
    newRange = scaleMax - scaleMin

    mat = (mat - oldMin) * newRange / oldRange + scaleMin
    return mat

In [ ]:
def getSymmetricMatrix(n):
    """
    Input:
    n - Integer
    
    Output: function returns a nXn symmetric matrix with random values
    """
    r = np.random.rand(n, n)
    F = (r + r.T)/2
    np.fill_diagonal(F,0)
    return F

def getF(attributes):
    """
    Input:
    attributes - Total number of attributes
    
    This function is used to get a initial Pheromone matrix with random values.
    
    Output:
    Function returns a matrix of shape :
    attributes X attributes X 4
    """
    F = np.zeros((attributes, attributes, 4))
    for i in range(4):
        F[:,:,i] = getSymmetricMatrix(attributes)
    return F

In [ ]:
def initialiseF(attributes):
    """
    Input:
    attributes - Total number of attributes
    
    This function is used to get a initial Pheromone matrix with heuristics.
    We use first method from methods of heuristics 
    to get Pheromone values for each link.
    
    Output:
    Function returns a matrix of shape :
    attributes X attributes X 4
    """
    F = np.zeros((attributes, attributes, 4))
    for i in range(attributes):
        for j in range(attributes):
            F[i][j][0] = heuristic(i, j, 0, 1)
            F[i][j][1] = heuristic(i, j, 1, 1)
            F[i][j][2] = heuristic(i, j, 2, 1)
            F[i][j][3] = heuristic(i, j, 3, 1)
    
    F = Scale(F, scaleMin, scaleMax)
    return F

In [ ]:
def heuristic(i, j, stateCode, methodCode):
    """
    Inputs:
    i, j -> From and To of the link whose heuristic importance is required.
    
    stateCode -> Specific code given to link based on which two states it connects.
    stateCode = 0 if link between state 0 to 0
              = 1 if link between state 0 to 1
              = 2 if link between state 1 to 0
              = 3 if link between state 1 to 1
    
    methodCode -> To spesify which method is to be used for heuristic calculations.
    methodCode = 1 for minimum redundancy method
               = 2 for minimum redundancy & maximum relevance method
               = 3 for Hybrid method
    
    Output: returns heuristic importance of the link
    """

    if methodCode == 1: #minimum redundancy
        if stateCode == 0:
            return corelationMat[i][j]
        if stateCode == 1:
            return (1 - corelationMat[i][j])
        if stateCode == 2:
            return corelationMat[i][j]
        if stateCode == 3:
            return (1 - corelationMat[i][j])

    if methodCode == 2: #minimum redundancy & maximum relevance
        if stateCode == 0:
            return np.sqrt(corelationMat[i][j] * (1 - relevanceVect[j]))
        if stateCode == 1:
            return np.sqrt(1 - corelationMat[i][j] * (relevanceVect[j]))
        if stateCode == 2:
            return np.sqrt(corelationMat[i][j] * (1 - relevanceVect[j]))
        if stateCode == 3:
            return np.sqrt(1 - corelationMat[i][j] * (relevanceVect[j]))
    
    if methodCode == 3: #Hybrid
        if stateCode == 0:
            return (1 - relevanceVect[j])
        if stateCode == 1:
            return (relevanceVect[j])
        if stateCode == 2:
            return corelationMat[i][j]
        if stateCode == 3:
            return 1 - corelationMat[i][j]

In [ ]:
def getAnts(attributes, noOfAnts, F):
    """
    Inputs:
    attributes - Total number of attributes
    noOfAnts   - number of ants
    F - Pheramone matrix, ndarray with shape [attributes X attributes X 4]
    
    Output:
    returns two lists of length equal to number of attributes,
    antsNode  -> list of attributes visited by ant
    antsState -> it gives states(o or 1) for curresponding nodes in antsNode
    """
    
    
    antsNode  = [np.random.choice(range(attributes), 1) for i in range(noOfAnts)]
    antsState = [np.random.choice(range(2),1) for i in range(noOfAnts)]
    
    for attribute in range(attributes-1):
        for ant in range(len(antsNode)):
            nextPossible = set(range(attributes)) - set(antsNode[ant])     
            i = antsNode[ant][-1] # last selected
            
# ------------------- Finding weights of possible attributes and states --------------------- #           
            
            if i: # last ant in state 1, it can either be 1 to 0, or 1 to 1.
                wt0 = {}
                for j in nextPossible:
                    HeuMethod = np.random.choice([1,2,3], p=[p1,p2,(1-(p1+p2))])
                    wt0[j] = F[i][j][2]**alpha * heuristic(i, j, 2, HeuMethod)**beta # state 1 to 0
                    
                wt1 = {}
                for j in nextPossible:
                    HeuMethod = np.random.choice([1,2,3], p=[p1,p2,(1-(p1+p2))])
                    wt1[j] = F[i][j][3]**alpha * heuristic(i, j, 3, HeuMethod)**beta # state 1 to 1

            else: # last ant in state 0, it can either be 0 to 0 or 0 to 1
                wt0 = {}
                for j in nextPossible:
                    HeuMethod = np.random.choice([1,2,3], p=[p1,p2,(1-(p1+p2))])
                    wt0[j] = F[i][j][0]**alpha * heuristic(i, j, 0, HeuMethod)**beta # state 0 to 0

                wt1 = {}
                for j in nextPossible:
                    HeuMethod = np.random.choice([1,2,3], p=[p1,p2,(1-(p1+p2))])
                    wt1[j] = F[i][j][1]**alpha * heuristic(i, j, 1, HeuMethod)**beta # state 0 to 1
            
# ------------ Exploration and Exploitation to select new attribute and state --------------- #           
            
            if np.random.rand(1) < 0.7 :#Exploitation
                select0 = max(wt0, key=wt0.get)
                select1 = max(wt1, key=wt1.get)

                if wt0[select0] > wt1[select1]:
                    antsNode[ant]  = np.append(antsNode[ant], select0)
                    antsState[ant] = np.append(antsState[ant], 0)
                else:
                    antsNode[ant]  = np.append(antsNode[ant], select1)
                    antsState[ant] = np.append(antsState[ant], 1)

            else:#Exploration
                Wt_wt0, Wt_wt1 = sum(wt0.values()), sum(wt1.values())
                P0 = Wt_wt0/(Wt_wt0 + Wt_wt1)
                P1 = Wt_wt1/(Wt_wt0 + Wt_wt1)
             
                if np.random.choice(range(2), p=[P0, P1]):
                    select1 = np.random.choice(list(nextPossible), 1, p = [wt1[i]/Wt_wt1 for i in nextPossible])
                    antsNode[ant] = np.append(antsNode[ant], select1)
                    antsState[ant] = np.append(antsState[ant], 1)

                else:            
                    select0 = np.random.choice(list(nextPossible), 1, p = [wt0[i]/Wt_wt0 for i in nextPossible])
                    antsNode[ant] = np.append(antsNode[ant], select0)
                    antsState[ant] = np.append(antsState[ant], 0)

    return antsNode, antsState

In [ ]:
def getAccuracy(xData, yData):
    """
    Given xData and yData i.e. attributes and target function runs RandomForestClassifier
    and returns 5 fold CV accuracy
    """

    kf = StratifiedKFold(n_splits=5)
    kf.get_n_splits()
    
    accuracy = []
    for train_index, valid_index in kf.split(xData, yData):
        
        X_Train = xData[train_index]
        y_Train = yData[train_index]
        X_Valid = xData[valid_index]
        y_Valid = yData[valid_index]

        rfModel = RandomForestClassifier()
        rfModel.fit(X_Train, y_Train)

        y_pred = rfModel.predict(X_Valid)
        accuracy.append(accuracy_score(y_Valid, y_pred))
               
    return np.mean(accuracy)

In [ ]:
def updateF(pMeasure, antsNode, antsState):
    """
    Inputs:
    pMeasure  -> list of performance score for ansts
    antsNode  -> list of nodes visited by ansts
    antsState -> list of states of nodes visited by ansts
    
    Given inputs this function calls modifyF with appropriate change fractions 
    which then modifies global variable F i.e. Pheromone maxtrix
    """
    global F
    
    evapFrac = 0.05
    evap = lambda x: x * (1-evapFrac)
    F = evap(F)
    
    best = np.argmax(pMeasure) 
    modifyF(antsNode[best], antsState[best], reward(pMeasure[best]*100) + evapFrac)
    
                   
def reward(accuracy):
    """
    Input:
    Accuracy in percentage
    
    Reward function returns increment fraction given accuracy value for 
    best performing ant
    """
    if accuracy <= 70:
        return 0.05
    if accuracy >= 90:
        return 0.30
    
    return (0.05 + (0.3 - 0.05)/(90 - 70) * (accuracy - 70))


def modifyF(antNode, antState, Change):
    """
    Inputs:
    antNode -  ant node list
    antState - ant state list corresponding to antnode list
    Change - change fraction in pheremone value 
    """
    for i in range(attributes-1):
        if antState[i] == 0 and antState[i+1] == 0:
            ind = 0

        if antState[i] == 0 and antState[i+1] == 1:
            ind = 1

        if antState[i] == 1 and antState[i+1] == 0:
            ind = 2

        if antState[i] == 1 and antState[i+1] == 1:
            ind = 3       

        F[antNode[i], antNode[i+1], ind] += F[antNode[i], antNode[i+1], ind]*Change
        F[antNode[i+1], antNode[i], ind] += F[antNode[i+1], antNode[i], ind]*Change

In [ ]:
def getSelectedFeatures(antNode, antState):
    """
    Inputs:
    antNode -  ant node list
    antState - ant state list corresponding to antnode list
    
    Output - List of selected attributes
    """
    attributeSelect = []
    for i in range(len(antNode)):
        if antState[i]:
            attributeSelect.append(antNode[i])
    return attributeSelect

In [ ]:
def output():
    dict = {
        'bestFSAccuracy':bestFSAccuracy[-1],
        'NoOfAttributes':len(bestSelect[-1]),
        'bestSelect':''.join(str(e)+" " for e in bestSelect[-1])
    }
    outputdf = pd.DataFrame(dict, index=[0])
    outputdf.to_csv(outputFile, mode='a', index=None,header=None)

# The main function

In [ ]:
def main(epoch):
    """
    Input:
    epoch -> Number of generations of ants to be generated.
    This function runs the Entire algorithm to get best feature set.
    """
    global F
    for itr in range(epoch):
        
        antsNode, antsState = getAnts(attributes, noOfAnts, F)
        accuracy = []
        learnMeasure = []
        attributeSelect = []
        
        for ant in range(noOfAnts):
            attributeSelect.append(getSelectedFeatures(antsNode[ant],antsState[ant]))

            if not len(attributeSelect[ant]) == 0:
                accu = getAccuracy(xData[:,attributeSelect[ant]], yData)
                accuracy.append(accu)     
            else:
                accuracy.append(0)
        
            learnMeasure.append(accuracy[ant]/(1 + lambd * len(attributeSelect[ant])))
            
        best = np.argmax(learnMeasure)
        if accuracy[best] > bestFSAccuracy[-1]:
            bestlearnMeasure.append(learnMeasure[best])
            bestFSAccuracy.append(accuracy[best])
            bestSelect.append(attributeSelect[best])
            print('best accuracy = ', round(accuracy[best], 4))
            print('best fitness score = ', round(learnMeasure[best],4))
            print('number of features selected = ', len(attributeSelect[best]))
                                              
        updateF(learnMeasure, antsNode, antsState)
        F = Scale(F, scaleMin, scaleMax)

        print('------ End of Epoch = ',itr+1,'-------') 
        

# Experiment

In [ ]:
def Experiment():
    """
    This function is built to repeat the feature selection algorithm multiple times 
    to test the statistical properties of results.
    
    Each time this function is called it intialises global variables, calles main()
    to get feature selection and writes the results in output file.
    """
    
    global F
    global bestFSAccuracy
    global bestSelect
    global incItr
    
    F = initialiseF(attributes)
    bestFSAccuracy = [0]
    bestSelect = [[]]
    incItr = [0]
    
    main(epoch)
    output()

# Select Data Set

In [ ]:
dset = datasets.load_breast_cancer()
dset_name = 'breast_cancer'

In [ ]:
xData = dset.data
yData = dset.target
attributes = xData.shape[1]

# Initialising Output File

In [ ]:
timestamp = str(datetime.now().replace(second=0, microsecond=0))
timestamp = timestamp.replace(' ','_')
outputFile = './output/'+ dset_name + '_' + timestamp +'.csv'

In [ ]:
dict = {
    1:'bestFSAccuracy',
    2:'NoOfAttributes',
    3:'bestSelect',
}
outputdf = pd.DataFrame(dict, index=[0])
outputdf.to_csv(outputFile, index=None, header=None)

# Computing Heuristics

## corelationMat

In [ ]:
scaleMin = 0.1
scaleMax = 0.9

In [ ]:
corelationMat = np.zeros((attributes, attributes))
for k in range(attributes):
    for l in range(attributes):
        corelationMat[k][l] = stats.pearsonr(xData[:,k], xData[:,l])[0]
corelationMat = np.abs(corelationMat)
print('max min', corelationMat.max(), corelationMat.min())
corelationMat = Scale(corelationMat, scaleMin, scaleMax)
print('max min', corelationMat.max(), corelationMat.min())

## relevanceVect

In [ ]:
rfModel = RandomForestClassifier()
rfModel.fit(xData, yData)

In [ ]:
gini = rfModel.feature_importances_
relevanceVect = Scale(gini, scaleMin, scaleMax)
print('max min', relevanceVect.max(), relevanceVect.min())

# Run

In [ ]:
F = None
bestlearnMeasure = [0]
bestFSAccuracy = [0]
bestSelect = [[]]

In [ ]:
Exrepeat = 5
noOfAnts = 10
epoch = 20

# alpha and beta are the parameters to give relative importance of learning(Pheromone) and Heuristics
alpha = 1
beta = 1

# lamda is the parameter to modify the objecive function to minimise number of feature selected while maximising accuracy
# higher value indicates mmore dominance of number of features on objecive function
lambd = 0.01

# p1, p2, (1-(p1+p2)) are the probabilities with which mothod 1, method 2 and method 3 of Heuristics are chosen
p1 = 0.33
p2 = 0.33

In [ ]:
for i in range(Exrepeat):
    print('====================== Experiment No =',i+1,'==========================')
    Experiment()

In [ ]:
result = pd.read_csv(outputFile) 
result